In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## CREATE FLAG PARAMETER

In [0]:
dbutils.widgets.text("incremental_flag","0","Incremental_flag")

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print(incremental_flag)

## CREATING DIMENSION

## fetch relative columns

In [0]:
df_src = spark.sql('''select distinct (Date_ID) as Date_ID
         from parquet.`abfss://silver@adlscardatalake.dfs.core.windows.net/carsales/`
         ''')

In [0]:
display(df_src)

##  dim date-sink Initial and incremental(Just bring the schema if table not exists)

In [0]:
if  spark.catalog.tableExists('cars_catalog.gold.dim_date'):
  df_sink = spark.sql('''select  dim_date_key,Date_ID
                    from cars_catalog.gold.dim_date
                    ''')
else:
  df_sink = spark.sql('''select 1 as dim_date_key,Date_ID
                    from  parquet.`abfss://silver@adlscardatalake.dfs.core.windows.net/carsales/` 
                    where 1= 0''')

In [0]:
df_filter = df_src.join(df_sink, df_src.Date_ID == df_sink.Date_ID, 'left').select(df_src.Date_ID,df_sink.dim_date_key)

In [0]:
df_filter.display()

**df_filter_old**

In [0]:
df_filter_old = df_filter.filter(df_filter.dim_date_key.isNotNull())

**df_filter_new**

In [0]:
df_filter_new = df_filter.filter(df_filter.dim_date_key.isNull()).select(df_src['Date_ID'])

In [0]:
df_filter_new.display()

## Create Surrogate key

**fetch the max surrogate key***

In [0]:
if (incremental_flag == '0') :
    max_value = 1
else:
   max_val_df = spark.sql("select max(dim_date_key) from cars_catalog.gold.dim_date")
   max_value = max_val_df.collect()[0][0]+1  #0th element of 0th row,get value from the dataframe
print(max_value)

***create surrorgate key and add the max surrogate key***


In [0]:
df_filter_new = df_filter_new.withColumn('dim_date_key', max_value + monotonically_increasing_id())

In [0]:
df_filter_new.display()

## create final_df = df_filter_old + df_filter_new

In [0]:
df_final = df_filter_old.union(df_filter_new)
df_final.display()


# SCD TYPE 1(UPSERT)

In [0]:
from delta.tables import DeltaTable

In [0]:
#source-df_Final,dest-dim_date

    #Incremental run
src = df_final

if spark.catalog.tableExists('cars_catalog.gold.dim_date'):
    delta_tbl = DeltaTable.forPath(spark,'abfss://gold@adlscardatalake.dfs.core.windows.net/dim_date')    #creating DeltaTable object on top of dim_date
    delta_tbl.alias('trg').merge (df_final.alias('src'), "trg.dim_date_key == src.dim_date_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
   #initial run
else:
    df_final.write.format('delta')\
        .mode('overwrite')\
        .option('path','abfss://gold@adlscardatalake.dfs.core.windows.net/dim_date')\
        .saveAsTable('cars_catalog.gold.dim_date')

In [0]:
%sql
select * from cars_catalog.gold.dim_date